In [1]:
import sys
!{sys.executable} -m pip install opencv-contrib-python --upgrade

In [2]:
import cv2
import numpy as np
import math

# Read the image
image = cv2.imread("./data/side.jpg", 1)
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
# Adapted from https://github.com/drakeguan/cp11fall_project1/blob/master/wlsFilter/wlsFilter.m
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve


def wls_filter(img_l, alpha=1.2, l=0.5, eps=sys.float_info.epsilon):

    img_l = img_l.astype(float)/255.0

    r, c = img_l.shape
    k = r*c

    # Compute affinities between adjacent pixels based on gradients of L
    dy = np.diff(img_l, 1, 0)
    dy = -l / (np.absolute(dy)**alpha + eps)
    dy = (np.vstack((dy, np.zeros(c, )))).flatten('F')

    dx = np.diff(img_l, 1, 1)
    dx = -l / (np.absolute(dx)**alpha + eps)
    dx = (np.hstack((dx, np.zeros(r, )[:, np.newaxis]))).flatten('F')

    # Construct a five-point spatially inhomogeneous Laplacian matrix
    B = np.vstack((dx, dy))
    d = [-r, -1]
    A = spdiags(B, d, k, k)

    e = dx
    w = np.roll(dx, r)
    s = dy
    n = np.roll(dy, 1)

    D = 1 - (e+w+s+n)
    A = A + A.T + spdiags(D, 0, k, k)

    # Solve
    OUT = spsolve(A, img_l.flatten('F')).reshape(img_l.shape[::-1])

    base = np.rollaxis(OUT, 1)
    detail = img_l - base
    return (base, detail)


C:\Users\gbata\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def shadow_enhance(img):
    # Perform edge-preserving base/detail decomposition
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    l, a, b = cv2.split(lab)
    (base, detail) = wls_filter(l)

    # Get saliency map
    # Based on https://pyimagesearch.com/2018/07/16/opencv-saliency-detection/
    base = (base*255).clip(0, 255).astype(np.uint8)
    saliency = cv2.saliency.StaticSaliencyFineGrained_create()
    (success, saliencyMap) = saliency.computeSaliency(img)
    cv2.imshow("Output", saliencyMap)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # compute DARK and BRIGHT
    # get f_sal to apply to new base
    dark = (l < 50) & (np.maximum.reduce(
        [l, a, b])-np.minimum.reduce([l, a, b]) > 5)
    d = l[dark].reshape(-1, 1)
    # Bug with getting base and details from DARK (sum of A with its transpose)
    #d_base,d_detail = wls_filter(d)
    bright = l[~dark]
    f_sal = min(2.0, (np.percentile(bright, 35))/(np.percentile(d, 95)))
    new_base = f_sal*saliencyMap*base + (1-saliencyMap)*base
    lab[:, :, 0] = (new_base+detail*255).clip(0, 255).astype(int)
    img_out = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)

    return img_out


In [5]:
img_out = shadow_enhance(image)


C:\Users\gbata\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


In [6]:
cv2.imshow("Final Result", np.hstack([image, img_out]))
cv2.waitKey(0)
cv2.destroyAllWindows()
